<a href="https://colab.research.google.com/github/myazann/Lung_Cancer/blob/main/Lung_Cancer_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' > /dev/null

!rm -rf sample_data
!rm sample.py
!rm README.md

!cp ./drive/MyDrive/bboxs.json .
!cp ./drive/MyDrive/lung_ct_train.zip .
!cp ./drive/MyDrive/lung_ct_val.zip .

!unzip lung_ct_train.zip > /dev/null
!rm lung_ct_train.zip
!unzip lung_ct_val.zip > /dev/null
!rm lung_ct_val.zip

In [1]:
import pandas as pd
import json
import os
import copy
import random
import numpy as np
import torchvision.transforms as transforms
import torch
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import itertools
import torchvision

from detectron2.structures import BoxMode
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

##Necessary functions



In [2]:
##Function to visualize an image with its corresponding label

def get_img_with_bbox(im, labels):

  d_im = copy.deepcopy(im.numpy().squeeze())

  xmax = int(labels["xmax"])
  xmin = int(labels["xmin"])
  ymax = int(labels["ymax"])
  ymin = int(labels["ymin"])

  cv2.rectangle(d_im, (xmin, ymin), (xmax, ymax), (255,255,255), 2)

  display(Image.fromarray(d_im))

## Create Image Dataset


In [3]:
def get_ct_scans(mode):

  category_map = {
                  "A": 0,
                  "B": 1,
                  "G": 2
                  }

  with open('bboxs.json') as json_file:
    bboxs = json.load(json_file)

  dataset_dicts = []
  path = "lung_ct_" + mode

  for ptnt in os.listdir(path):
    for i, img in enumerate(os.listdir(path + "/" + ptnt)):

      label = {}
      bboxes = bboxs[img.split(".j")[0]]

      label["filename"] = path + "/" + ptnt + "/" + img
      label["image_id"] = i
      label["height"] = 512
      label["width"] = 512
      boxes = {
      "bbox": [int(bboxes["xmin"]), int(bboxes["ymin"]), int(bboxes["xmax"]), int(bboxes["ymax"])],
      "bbox_mode": BoxMode.XYXY_ABS,
      "category_id": category_map[bboxes["class"]],
      }
      label["annotations"] = boxes

      dataset_dicts.append(label)  

  return dataset_dicts

In [4]:
for d in ["train", "val"]:
  DatasetCatalog.register("ct_" + d, lambda d=d: get_ct_scans(d))
  MetadataCatalog.get("ct_" + d).set(thing_classes=["A", "B", "G"])

In [ ]:
DatasetCatalog.get("ct_train")

##See randomly selected 10 images

In [ ]:
rs = random.sample(range(0, len(ct_train)), 10)

for elem in rs:
  get_img_with_bbox(ct_train[elem][0], ct_train[elem][1])
  print("\n")

In [ ]:
"""
    label["img_name"] = self.imgs[index]
    label["image_id"] = index
    label["height"] = 512
    label["width"] = 512
    boxes = {
    "bbox": [item_label["xmin"], item_label["ymin"], item_label["xmax"], item_label["ymax"]],
    "bbox_mode": BoxMode.XYXY_ABS,
    "category_id": self.category_map[item_label["class"]],
    }
    label["annotations"] = obj
    
"""

In [14]:
class CTDataset(torch.utils.data.Dataset):

  def __init__(self, train=True, transform=True):

    self.transform = transform 
    self.train = train
    self.imgs = get_image_paths(self.train)
    self.labels = bboxs  
    self.category_map = {
                          "A": 0,
                          "B": 1,
                          "G": 2
                        }
        
  def __getitem__(self, index):

    im = cv2.imread(self.imgs[index])
    im = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

    im = torch.tensor(im, dtype=torch.double)[None, ...]
    if self.transform:
      im = self.preprocess(im)

    img_name = self.imgs[index].split("/")[-1].split(".j")[0]
    item_label = self.labels[img_name]

    label = {}
    label["img_name"] = self.imgs[index]
    label["image_id"] = index
    label["height"] = 512
    label["width"] = 512
    boxes = {
    "bbox": [item_label["xmin"], item_label["ymin"], item_label["xmax"], item_label["ymax"]],
    "bbox_mode": BoxMode.XYXY_ABS,
    "category_id": self.category_map[item_label["class"]],
    }
    label["annotations"] = boxes

    return im, label

  def __len__(self):
    return len(self.imgs)

  def preprocess(self, img):

      return transforms.Compose(
      [
      ##transforms.ToTensor(),
      transforms.CenterCrop(384),
      transforms.RandomHorizontalFlip(0.5),
      transforms.RandomRotation(30),
      transforms.RandomVerticalFlip(),
      transforms.GaussianBlur(3),
      transforms.Normalize((0.,), (1.,))
      ]
      )(img)